In [ ]:
# Data manipulation
import pandas as pd
import numpy as np

# Plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Finance related operations
from pandas_datareader import data

# Import this to silence a warning when converting data column of a dataframe on the fly
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [ ]:
# Load data
df = pd.read_csv('../input/df-out1/df_out.csv', index_col=0)

df.head()

In [ ]:
df_2018 = pd.read_csv('../input/200-financial-indicators-of-us-stocks-20142018/2018_Financial_Data.csv', index_col=0)
df_2018.head()

In [ ]:
df_2018 = df_2018.rename(columns={'Enterprise Value': 'Enterprise Value 2018'})
df_2018x = df_2018[df_2018['Enterprise Value 2018'].notna()]
df_2018x = df_2018x['Enterprise Value 2018']

In [ ]:
df_1418 = pd.merge(df, df_2018x, right_index =True, left_index = True, how='inner')
df_1418.head()

In [ ]:
df_2014 = pd.read_csv('../input/200-financial-indicators-of-us-stocks-20142018/2014_Financial_Data.csv', index_col=0)
EV_2014 = df_2014['Enterprise Value']
df_1418x = pd.merge(df_1418, EV_2014, right_index =True, left_index = True, how='inner')
df_1418x.head()

In [ ]:
df_1418x['Growth Rate'] = df_1418x['Enterprise Value 2018'] / df_1418x['Enterprise Value']
df_1418x.head()
df_1418x = df_1418x.dropna()

In [ ]:
##category = pd.cut(df_1418x['Growth Rate'],bins=[-np.inf,0,1.461,np.inf],labels=[-1,1,10])
##category = pd.cut(df_1418x['Growth Rate'],bins=[-np.inf,0,np.inf],labels=[-1,1])
category = pd.cut(df_1418x['Growth Rate'],bins=[-np.inf,2,np.inf],labels=['NG','G'])
df_1418x.insert(67,'Stock Performance',category)

In [ ]:
df_1418x['Stock Performance'].value_counts(normalize=True)

In [ ]:
features2 = ['Revenue', 'EPS', 'EBITDA Margin', 'returnOnEquity', 'EBIT Growth', 'Operating Income Growth', 'Sector']
df_1418x[(df_1418x['Stock Performance'] == 'G') & (df_1418x['Sector'] == 'Technology')][features2].describe()

In [ ]:
df_1418x[df_1418x['Stock Performance'] == 'G']['Sector'].value_counts(normalize=True)

In [ ]:
df_1418x[(df_1418x['Stock Performance'] == 'NG') & (df_1418x['Sector'] == 'Technology')][features2].describe()

In [ ]:
df_1418x[df_1418x['Stock Performance'] == 'NG']['Sector'].value_counts(normalize=True)

In [ ]:
features = ['Revenue', 'EPS', 'EBITDA Margin', 'returnOnEquity', 'Operating Income Growth', 'Sector']
X = df_1418x[features]
y = df_1418x.loc[:, df_1418x.columns.intersection(['Stock Performance'])]

In [ ]:
one_hot_X = pd.get_dummies(X)
one_hot_X.replace([np.inf, -np.inf], np.nan)
one_hot_X.fillna(0)
one_hot_X.head()

In [ ]:
y = y.values.ravel()
print(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(one_hot_X, y, test_size=0.25, random_state=52)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


# Define the model. Set random_state to 1
rf_model = forest_model = RandomForestRegressor(random_state=1)

# fit your model
rf_model.fit(X_train, y_train)

# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(y_test, rf_model.predict(X_test))

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))



print(rf_model.predict(X_test))

results = rf_model.predict(X_test)

print(X_test)

X_test.insert(1,'Actual', y_test)
print(X_test)

results1 = pd.DataFrame({'Results': results}, index = X_test.index)
print(results1)

X_test.insert(2, 'Prediction', results1['Results'])
print(X_test)

from sklearn.tree import export_graphviz

export_graphviz(rf_model.estimators_[0], out_file='tree.dot', feature_names=X.columns, filled=True, rounded=True)

!dot -Tpng tree.dot -o tree.png

from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    
    log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [ ]:
sns.set_color_codes("muted")
sns.barplot(x='Accuracy', y='Classifier', data=log, color="b")

plt.xlabel('Accuracy %')
plt.title('Classifier Accuracy')
plt.show()

sns.set_color_codes("muted")
sns.barplot(x='Log Loss', y='Classifier', data=log, color="g")

plt.xlabel('Log Loss')
plt.title('Classifier Log Loss')
plt.show()

In [ ]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()